In [19]:
import pandas as pd
import re
import numpy as np


In [42]:
data = pd.read_csv('output.csv')
df = pd.DataFrame(data)
df

,Title,Phone,Website,Street Address,City,State,Zip,County,Specialty,Hours,Accommodations,Telehealth Offered,Languages/Cultural Capabilities,Language Line Interpretation Services Available,Accepting New Clients
0,CAMINAR COLMA RIDGE,(650) 994-3582,http://www.caminar.org,85 REINER STREET,DALY CITY,CA,94014,San Mateo,Mental Health Clinic/Center (Including Communi...,Monday through Friday 08:00 AM to 05:00 PM\t\t...,This location is handicapped accessible (ADA C...,Services at this site are provided both in-per...,Spanish:Good;Vietnamese:Good,Yes,Yes
1,CAMINAR EUCALYPTUS HOUSE,(650) 994-7110,http://www.caminar.org,2 EDGEWOOD COURT,DALY CITY,CA,94014,San Mateo,"Community/Behavioral Health Agency, Non-Indivi...",Monday through Sunday 12:00 AM to 11:59 PM\t\t...,This location is handicapped accessible (ADA C...,Services at this site are provided both in-per...,Spanish:Good;Tagalog:Good;MandarinChinese:Good,Yes,Yes
2,CAMINAR HAWTHORNE HOUSE,(650) 368-2383,http://www.caminar.org,251 JACKSON AVENUE,REDWOOD CITY,CA,94061,San Mateo,"Community/Behavioral Health Agency, Non-Indivi...",Monday through Sunday 12:00 AM to 11:59 PM\t\t...,This location is handicapped accessible (ADA C...,Services at this site are provided both in-per...,Spanish:Good;Tagalog:Good,Yes,Yes
3,CAMINAR REDWOOD HOUSE,(650) 839-1076,http://www.caminar.org,248 REDWOOD AVENUE,REDWOOD CITY,CA,94061,San Mateo,Mental Illness Community Based Residential Tre...,Monday through Sunday 12:00 AM to 11:59 PM\t\t...,This location is handicapped accessible (ADA C...,Services at this site are provided both in-per...,Spanish:Good;Tagalog:Good;AmericanSignLanguage...,Yes,Yes
4,CAMINAR YAIL,(650) 367-9610,http://www.caminar.org,37 CLINTON STREET,REDWOOD CITY,CA,94062,San Mateo,Mental Health Clinic/Center (Including Communi...,Monday through Friday 09:00 AM to 05:00 PM\t\t...,This location is handicapped accessible (ADA C...,Services at this site are provided both in-per...,Spanish:Good,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,SERVICE LEAGUE OF SAN MATEO COUNTY - HOPE HOUSE,(650) 363-8735,https://serviceleague.org/get-help/hope-house-...,3789 HOOVER ST,REDWOOD CITY,CA,94063,San Mateo,"Substance Abuse Rehabilitation Facility, Non-I...",Monday through Friday 09:00 AM to 05:00 PM\t\t...,This location is less than 5 blocks from publi...,No provider uses telehealth at this site,Spanish:Good,Yes,Yes
81,FORT HELP NARCOTICS REPLACEMENT THERAPY - BRYANT,(415) 777-9953,https://www.americanhealthservices.org/san-fra...,915 BRYANT ST,SAN FRANCISCO,CA,94103,San Francisco,"Methadone Clinic, Non-Individual - Ambulatory ...",Monday through Friday 06:15 AM to 01:45 PMSatu...,This location is less than 1 block from public...,Services at this site are provided both in-per...,Spanish:Good,Yes,Yes
82,FORT HELP NARCOTICS REPLACEMENT THERAPY- CAPP ...,(415) 821-1427,https://www.americanhealthservices.org/san-fra...,1101 CAPP ST,SAN FRANCISCO,CA,94110,San Francisco,"Methadone Clinic, Non-Individual - Ambulatory ...",Monday through Friday 06:00 AM to 02:30 PMSatu...,This location is less than 1 block from public...,Services at this site are provided both in-per...,Spanish:Good,Yes,Yes
83,LIFELINE TREATMENT SERVICES,(510) 777-8448,https://www.americanhealthservices.org/oakland...,10429 INTERNATIONAL BLVD,OAKLAND,CA,94603,Alameda,"Methadone Clinic, Non-Individual - Ambulatory ...",Monday through Friday 06:00 AM to 02:30 PMSatu...,This location is less than 1 block from public...,Services at this site are provided both in-per...,Spanish:Good,Yes,Yes


In [69]:
ada_compliant = []
deaf = []
block_pub_transport = []
mile_pub_transport = []
telehealth = []


language_dict = {'Spanish': [],
 'Vietnamese': [],
 'Tagalog': [],
 'MandarinChinese': [],
 'AmericanSignLanguage': [],
 'YueChinese': [],
 'Russian': [],
 'Korean': [],
 'Persian': [],
 'Chinese': [],
 'Armenian': [],
 'Japanese': [],
 'Arabic': [],
 'Khmer': [],
 'Hmong': [],
 'Italian': [],
 'Samoan': [],
 'Ukrainian': [],
 'Portuguese': [],
 'Panjabi': []
 }

serves_children = []
serves_adults =[]
hours = []

for ind, row in df.iterrows():
    ACCOMODATIONS = row['Accommodations']

    if 'This location is handicapped accessible (ADA Compliant)' in ACCOMODATIONS:
        ada_compliant.append(1)
    else:
        ada_compliant.append(0)
    if 'This location has Telecommunication Device for the Deaf (TDD) equipment' in ACCOMODATIONS:
        deaf.append(1)
    else:
        deaf.append(0)
    if 'from public transportation' in ACCOMODATIONS:
        block_pattern = r"less than (\d+) block"
        mile_pattern = r"less than (\d+) mile"

        block_match = re.search(block_pattern, ACCOMODATIONS)
        mile_match = re.search(mile_pattern, ACCOMODATIONS)


        if block_match:
            number_of_blocks = int(block_match.group(1)) 
            block_pub_transport.append(number_of_blocks)
            mile_pub_transport.append(np.nan)
        else:
            block_pub_transport.append(np.nan) # represents that it is not specified
            if mile_match:
                number_of_miles = int(mile_match.group(1)) 
                mile_pub_transport.append(number_of_miles)
            else:
                mile_pub_transport.append(np.nan)

    TELEHEALTH = row['Telehealth Offered']
    if 'Services at this site are provided both in-person and through telehealth' in TELEHEALTH:
        telehealth.append(1)
    else:
        telehealth.append(0)

    LANGUAGE = row['Languages/Cultural Capabilities']
  
    if type(LANGUAGE) == float:
        for lang in language_dict.keys():
            language_dict[lang].append('')
    else: 
        language_pairs = LANGUAGE.split(';')
    
        longest_list = len(language_dict['Spanish'])
        for language_pair in language_pairs:
            language = language_pair.split(':')[0]
            level = language_pair.split(':')[1]
            language_dict[language].append(level)
            longest_list = len(language_dict[language]) #now we use this to compare to other lists

        for otherlang in language_dict.keys():
            if len(language_dict[otherlang]) != longest_list:
                language_dict[otherlang].append('')


    HOURS = row['Hours']
    

    both = r"Age Groups Served:Site Serves Children and Adults"
    children = r"Age Groups Served:Site Serves only Children"
    adults = r"Age Groups Served:Site Serves only Adults"
    
    
    if re.search(both, HOURS):
        serves_children.append(1)
        serves_adults.append(1)
    else:
        if re.search(children, HOURS):
            serves_children.append(1)
            serves_adults.append(0)
        else:
            if re.search(adults, HOURS):
                serves_children.append(0)
                serves_adults.append(1)
 

    # HOURS
    last_pm_index = HOURS.rfind("PM")
    if last_pm_index != -1:
        # Remove the portion starting from the last "PM"
        modified_text = HOURS[:last_pm_index + 2].strip()
        hours.append(modified_text)


In [70]:
df['ADA Compliant'] = ada_compliant
df['Telecommunication Device for the Deaf (TDD) equipment'] = deaf
df['Blocks from public tranportation'] = block_pub_transport
df['Miles from public tranportation'] = mile_pub_transport
for lang in language_dict.keys():
    df[lang] = language_dict[lang]

df['Serves Children'] = serves_children
df['Serves Adults'] = serves_adults
df['Open Hours'] = hours


In [71]:
df.to_csv('new.csv')
